In [12]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)
import os
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv("../src/core/en_azure/.env")

True

In [13]:
search_endpoint: str = os.getenv("SEARCH_ENDPOINT")
search_api_key: str = os.getenv("SEARCH_KEY")
index_name: str = os.getenv("SEARCH_NAME")

In [14]:
documents=[
    {
        "@search.action": "upload",
        "id": "1",
        "FormName": "Medical Insurance - Hospitalization & Surgical Claim Form",
        "FormDescription": "This claim form is tailored for customers seeking reimbursement for medical expenses due to hospitalization or surgical procedures. To be used when the claimed amount does not exceed HKD 3000. The form requires detailed information on the medical services received, along with proof of expense. Policyholders should ensure that all sections are completed accurately to facilitate prompt processing.",
        "FormCategory": "Claims",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/medical-insurance-hospitalization-surgical-claim-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "2",
        "FormName": "Voluntary Health Insurance Scheme (VHIS) Claimable Amount Estimate / Preliminary Assessment",
        "FormDescription": "This important form serves as an estimate or preliminary assessment document for policyholders under the Voluntary Health Insurance Scheme (VHIS). It aids in understanding the potential claimable amounts under the scheme's provisions. By providing preliminary information, customers can better plan their healthcare finances. Timely submission of the form enables smoother claim processing.",
        "FormCategory": "Claims",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/vhis-claimable-amount-estimate-preliminary-assessment-request-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "3",
        "FormName": "OGD/Colonoscopy Day Surgery or Hospital Admission Consultation Form",
        "FormDescription": "Designed for individuals undergoing an OGD, colonoscopy, or other related day surgeries, this form must be submitted for hospital admission consultation. It captures specific details about the procedure to ensure proper coverage and claims processing. The form plays a crucial role in the administrative preparation for both planned and emergency admissions. Completion of this form streamlines patient admission and insurance procedures.",
        "FormCategory": "Claims",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/ogd-colonoscopy-day-surgery-or-hospital-admission-consultation-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "4",
        "FormName": "Group Medical Claim Form",
        "FormDescription": "This comprehensive form is intended for use by organizations to submit medical claims on behalf of their members or employees. It is applicable when the total claim amount exceeds HKD 3000. The form requires detailed group policy information, individual claimant details, and specifics regarding the medical services availed. Proper completion of this form enables efficient claim settlement for group policies.",
        "FormCategory": "Claims",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/group-medical-claim-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "5",
        "FormName": "Change of Contact Details Form",
        "FormDescription": "A convenient option for policyholders needing to update their contact information including mailing addresses, email, and telephone numbers. This form is critical for maintaining clear and uninterrupted communication between the policyholder and the insurer. Accurate and current contact details ensure that policyholders receive timely updates and correspondence. It is also essential for the security of policy-related information.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/cs01a-change-of-contact-details-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "6",
        "FormName": "Change of Personal Particulars Form",
        "FormDescription": "Helps individual policyholders keep their personal information such as names, legal identification numbers, birth dates, and gender up to date with the insurer. Keeping personal information current is critical for the validation of the insurance agreement and related services. This form must be accurately filled out to reflect any legal changes to personal particulars. It also ensures the policy remains in force without unnecessary complications.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/cs01b-change-of-personal-particulars-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "7",
        "FormName": "Joint Owner Authorization Form",
        "FormDescription": "This vital form enables joint policy owners to grant authorization for accessing online services related to their Manulife policy. It is mandatory for all joint owners to provide their consent via signatures to effect any changes. This multilateral agreement is critical for processing requests and ensuring joint decisions. Proper completion of this form is essential for maintaining the shared rights and privileges of each policy owner.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/joint-owner-authorization-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "8",
        "FormName": "Customer Consent to Use / Provision of Personal Data in Direct Marketing",
        "FormDescription": "Policyholders who wish to allow or restrict the use of their personal data for direct marketing activities can do so with this form. Consent is given for Manulife to curate specific financial products and services that align with customer interests. A policyholder's choice regarding marketing preferences is respected and can be changed at any time. This document ensures compliance with personal data protection laws and customer-centric marketing ethics.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/cs03-customer-consent-to-use-provision-of-personal-data-in-direct-marketing.pdf"
    },
    {
        "@search.action": "upload",
        "id": "9",
        "FormName": "Change Premium Payment Mode/ Method",
        "FormDescription": "Policyholders looking to make amendments to their premium payment schedule or method can use this form to submit their request. Whether changing from monthly to annual payments or from direct debit to credit card, this form facilitates the process. The flexibility of premium payment is important for adapting to policyholder's changing financial situations. Timely updates help prevent policy lapses due to payment issues.",
        "FormCategory": "Policy Management",
        "url": "https://individuallogin.manulife.com.hk/thf/auth/signIn?lang=en_US"
    },
    {
        "@search.action": "upload",
        "id": "10",
        "FormName": "Beneficiary Designation Form",
        "FormDescription": "This crucial document allows policyholders to designate or change the nominee(s) who will receive the benefits of the policy upon the policyholder's demise. A clear beneficiary designation is critical for the proper execution of the policyholder's wishes and for ensuring financial security to the chosen beneficiaries. The form requires thoughtful consideration and precision to prevent disputes. Policyholders are encouraged to update this form after major life events.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/t02b-fillable-beneficiary-designation-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "11",
        "FormName": "Change of Policyowner Personal Details Form",
        "FormDescription": "A tool for existing policyowners to formally update their personal details on record with the insurance company. Changes might be necessitated by events such as marriage (change in surname), relocation, or any legal change in personal information. Keeping personal details accurate is vital for the policy's integrity and for seamless service delivery. The form also aids in the authentication of the policyowner's identity.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/change-of-policyowner-personal-details-fillable.pdf"
    },
    {
        "@search.action": "upload",
        "id": "12",
        "FormName": "Risk Profile Questionnaire",
        "FormDescription": "A critical self-assessment tool helping policyholders and financial advisors determine an individual's investment risk tolerance and financial goals. It guides the selection of suitable investment-linked insurance products and strategies. By regularly reviewing and updating risk preferences on this questionnaire, policyholders can align their portfolios with their current risk appetite. This dynamic assessment aids in informed decision-making regarding investment choices.",
        "FormCategory": "Policy Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/risk-profile-questionnaire.pdf"
    },
    {
        "@search.action": "upload",
        "id": "13",
        "FormName": "Manulife Investment Plus 2 (Hong Kong) Investment Choice Table",
        "FormDescription": "Serves as a comprehensive guide to investment options available under Manulife Investment Plus and related products. It provides critical information on the various fund choices, performance metrics, and risk assessments. Policyholders use this table for making informed decisions about allocating their premiums to different investment funds. Regular consultation of this table is recommended for strategic portfolio adjustments.",
        "FormCategory": "Fund Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/investment-choice-table-manulife-investment-solutions.pdf"
    },
    {
        "@search.action": "upload",
        "id": "14",
        "FormName": "Manulife Investment Plus 2 (Hong Kong) Application For Payout Distribution Investment Choice",
        "FormDescription": "Policyholders who wish to apply for payouts or distributions related to their investment-linked assurance schemes will require this form. It specifies how and when distributions are to be invested or received. The form is instrumental in helping customers manage their investment returns and financial planning. Accurate completion ensures that requests for payouts are processed according to the policyholder's needs.",
        "FormCategory": "Fund Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/applicant-for-payout-distribution-investment-choice.pdf"
    },
    {
        "@search.action": "upload",
        "id": "15",
        "FormName": "Manulife Investment Plus 2 (Hong Kong) Investment Choice Services Form",
        "FormDescription": "A multi-purpose form designed for policyholders to initiate changes related to their investment-linked policies such as top-up premiums, fund switching, redemptions, and full surrender. Making adjustments to one's investment strategy is critical for adapting to the changing market conditions or personal circumstances. This form facilitates the smooth execution of these transactional services. Policyholders are encouraged to seek financial advice when executing substantial changes.",
        "FormCategory": "Fund Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/mip2-investment-choice-services-form.pdf"
    },
    {
        "@search.action": "upload",
        "id": "16",
        "FormName": "Manulife Investment Plus 2 (Hong Kong) Policy Change Form",
        "FormDescription": "This document grants policyholders the ability to request various policy changes related to the Manulife Investment Plus 2 plans. Changes can range from amendments to personal details, fund allocation, to beneficiary designations. The form enables rapid adjustments to ensure policy details remain aligned with the customer's current needs and wishes. Proper and prompt completion of the policy change form is crucial in maintaining policy accuracy and efficacy.",
        "FormCategory": "Fund Management",
        "url": "https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/policy-change-form.pdf"
    },
]

In [15]:
documents = [{'@search.action': 'upload',
  'id': '1',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/medical-insurance-hospitalization-surgical-claim-form.pdf',
  'title': 'Medical Insurance - Hospitalization & Surgical Claim Form',
  'content': 'This claim form is tailored for customers seeking reimbursement for medical expenses due to hospitalization or surgical procedures. To be used when the claimed amount does not exceed HKD 3000. The form requires detailed information on the medical services received, along with proof of expense. Policyholders should ensure that all sections are completed accurately to facilitate prompt processing.',
  'category': 'Claims'},
 {'@search.action': 'upload',
  'id': '2',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/vhis-claimable-amount-estimate-preliminary-assessment-request-form.pdf',
  'title': 'Voluntary Health Insurance Scheme (VHIS) Claimable Amount Estimate / Preliminary Assessment',
  'content': "This important form serves as an estimate or preliminary assessment document for policyholders under the Voluntary Health Insurance Scheme (VHIS). It aids in understanding the potential claimable amounts under the scheme's provisions. By providing preliminary information, customers can better plan their healthcare finances. Timely submission of the form enables smoother claim processing.",
  'category': 'Claims'},
 {'@search.action': 'upload',
  'id': '3',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/ogd-colonoscopy-day-surgery-or-hospital-admission-consultation-form.pdf',
  'title': 'OGD/Colonoscopy Day Surgery or Hospital Admission Consultation Form',
  'content': 'Designed for individuals undergoing an OGD, colonoscopy, or other related day surgeries, this form must be submitted for hospital admission consultation. It captures specific details about the procedure to ensure proper coverage and claims processing. The form plays a crucial role in the administrative preparation for both planned and emergency admissions. Completion of this form streamlines patient admission and insurance procedures.',
  'category': 'Claims'},
 {'@search.action': 'upload',
  'id': '4',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/group-medical-claim-form.pdf',
  'title': 'Group Medical Claim Form',
  'content': 'This comprehensive form is intended for use by organizations to submit medical claims on behalf of their members or employees. It is applicable when the total claim amount exceeds HKD 3000. The form requires detailed group policy information, individual claimant details, and specifics regarding the medical services availed. Proper completion of this form enables efficient claim settlement for group policies.',
  'category': 'Claims'},
 {'@search.action': 'upload',
  'id': '5',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/cs01a-change-of-contact-details-form.pdf',
  'title': 'Change of Contact Details Form',
  'content': 'A convenient option for policyholders needing to update their contact information including mailing addresses, email, and telephone numbers. This form is critical for maintaining clear and uninterrupted communication between the policyholder and the insurer. Accurate and current contact details ensure that policyholders receive timely updates and correspondence. It is also essential for the security of policy-related information.',
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '6',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/cs01b-change-of-personal-particulars-form.pdf',
  'title': 'Change of Personal Particulars Form',
  'content': 'Helps individual policyholders keep their personal information such as names, legal identification numbers, birth dates, and gender up to date with the insurer. Keeping personal information current is critical for the validation of the insurance agreement and related services. This form must be accurately filled out to reflect any legal changes to personal particulars. It also ensures the policy remains in force without unnecessary complications.',
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '7',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/joint-owner-authorization-form.pdf',
  'title': 'Joint Owner Authorization Form',
  'content': 'This vital form enables joint policy owners to grant authorization for accessing online services related to their Manulife policy. It is mandatory for all joint owners to provide their consent via signatures to effect any changes. This multilateral agreement is critical for processing requests and ensuring joint decisions. Proper completion of this form is essential for maintaining the shared rights and privileges of each policy owner.',
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '8',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/cs03-customer-consent-to-use-provision-of-personal-data-in-direct-marketing.pdf',
  'title': 'Customer Consent to Use / Provision of Personal Data in Direct Marketing',
  'content': "Policyholders who wish to allow or restrict the use of their personal data for direct marketing activities can do so with this form. Consent is given for Manulife to curate specific financial products and services that align with customer interests. A policyholder's choice regarding marketing preferences is respected and can be changed at any time. This document ensures compliance with personal data protection laws and customer-centric marketing ethics.",
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '9',
  'url': 'https://individuallogin.manulife.com.hk/thf/auth/signIn?lang=en_US',
  'title': 'Change Premium Payment Mode/ Method',
  'content': "Policyholders looking to make amendments to their premium payment schedule or method can use this form to submit their request. Whether changing from monthly to annual payments or from direct debit to credit card, this form facilitates the process. The flexibility of premium payment is important for adapting to policyholder's changing financial situations. Timely updates help prevent policy lapses due to payment issues.",
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '10',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/t02b-fillable-beneficiary-designation-form.pdf',
  'title': 'Beneficiary Designation Form',
  'content': "This crucial document allows policyholders to designate or change the nominee(s) who will receive the benefits of the policy upon the policyholder's demise. A clear beneficiary designation is critical for the proper execution of the policyholder's wishes and for ensuring financial security to the chosen beneficiaries. The form requires thoughtful consideration and precision to prevent disputes. Policyholders are encouraged to update this form after major life events.",
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '11',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/change-of-policyowner-personal-details-fillable.pdf',
  'title': 'Change of Policyowner Personal Details Form',
  'content': "A tool for existing policyowners to formally update their personal details on record with the insurance company. Changes might be necessitated by events such as marriage (change in surname), relocation, or any legal change in personal information. Keeping personal details accurate is vital for the policy's integrity and for seamless service delivery. The form also aids in the authentication of the policyowner's identity.",
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '12',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/risk-profile-questionnaire.pdf',
  'title': 'Risk Profile Questionnaire',
  'content': "A critical self-assessment tool helping policyholders and financial advisors determine an individual's investment risk tolerance and financial goals. It guides the selection of suitable investment-linked insurance products and strategies. By regularly reviewing and updating risk preferences on this questionnaire, policyholders can align their portfolios with their current risk appetite. This dynamic assessment aids in informed decision-making regarding investment choices.",
  'category': 'Policy Management'},
 {'@search.action': 'upload',
  'id': '13',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/investment-choice-table-manulife-investment-solutions.pdf',
  'title': 'Manulife Investment Plus 2 (Hong Kong) Investment Choice Table',
  'content': 'Serves as a comprehensive guide to investment options available under Manulife Investment Plus and related products. It provides critical information on the various fund choices, performance metrics, and risk assessments. Policyholders use this table for making informed decisions about allocating their premiums to different investment funds. Regular consultation of this table is recommended for strategic portfolio adjustments.',
  'category': 'Fund Management'},
 {'@search.action': 'upload',
  'id': '14',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/applicant-for-payout-distribution-investment-choice.pdf',
  'title': 'Manulife Investment Plus 2 (Hong Kong) Application For Payout Distribution Investment Choice',
  'content': "Policyholders who wish to apply for payouts or distributions related to their investment-linked assurance schemes will require this form. It specifies how and when distributions are to be invested or received. The form is instrumental in helping customers manage their investment returns and financial planning. Accurate completion ensures that requests for payouts are processed according to the policyholder's needs.",
  'category': 'Fund Management'},
 {'@search.action': 'upload',
  'id': '15',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/mip2-investment-choice-services-form.pdf',
  'title': 'Manulife Investment Plus 2 (Hong Kong) Investment Choice Services Form',
  'content': "A multi-purpose form designed for policyholders to initiate changes related to their investment-linked policies such as top-up premiums, fund switching, redemptions, and full surrender. Making adjustments to one's investment strategy is critical for adapting to the changing market conditions or personal circumstances. This form facilitates the smooth execution of these transactional services. Policyholders are encouraged to seek financial advice when executing substantial changes.",
  'category': 'Fund Management'},
 {'@search.action': 'upload',
  'id': '16',
  'url': 'https://www.manulife.com.hk/content/dam/insurance/hk/en/documents/services/forms/policy-change-form.pdf',
  'title': 'Manulife Investment Plus 2 (Hong Kong) Policy Change Form',
  'content': "This document grants policyholders the ability to request various policy changes related to the Manulife Investment Plus 2 plans. Changes can range from amendments to personal details, fund allocation, to beneficiary designations. The form enables rapid adjustments to ensure policy details remain aligned with the customer's current needs and wishes. Proper and prompt completion of the policy change form is crucial in maintaining policy accuracy and efficacy.",
  'category': 'Fund Management'}]

In [16]:
# def change_key(dict_obj, old_key, new_key):
#     if old_key in dict_obj:
#         dict_obj[new_key] = dict_obj.pop(old_key)  # Using pop() not only returns the value but also removes the old key


In [17]:
# # A dictionary mapping old key names to new key names
# keys_to_rename = {
#     'FormName': 'title',
#     'FormDescription': 'content',
#     'FormCategory': 'category'
# }

# for doc in documents:
#     # Iterate over the mappings and rename the keys
#     for old_key, new_key in keys_to_rename.items():
#         if old_key in doc:
#             doc[new_key] = doc.pop(old_key)

In [18]:
def get_embedding(text, embedding_model_endpoint=None, embedding_model_key=None, azure_credential=None):
    endpoint = embedding_model_endpoint if embedding_model_endpoint else os.getenv("EMBEDDING_MODEL_ENDPOINT")
    key = embedding_model_key if embedding_model_key else os.getenv("EMBEDDING_MODEL_KEY")
    
    if azure_credential is None and (endpoint is None or key is None):
        raise Exception("EMBEDDING_MODEL_ENDPOINT and EMBEDDING_MODEL_KEY are required for embedding")

    try:
        endpoint_parts = endpoint.split("/openai/deployments/")
        base_url = endpoint_parts[0]
        #print(base_url)
        deployment_id = endpoint_parts[1].split("/embeddings")[0]
        #print(deployment_id)

        api_version = endpoint_parts[1].split("api-version=")[1].split("&")[0]

        # if azure_credential is not None:
        #     api_key = azure_credential.get_token("https://cognitiveservices.azure.com/.default").token
        # else:
        api_key = key

        client = AzureOpenAI(api_version=api_version, azure_endpoint=base_url, api_key=api_key)
        embeddings = client.embeddings.create(model=deployment_id, input=text)
        return embeddings.data[0].embedding

    except Exception as e:
        raise Exception(f"Error getting embeddings with endpoint={endpoint} with error={e}")

In [19]:
for doc in documents:
    doc["vector"] = get_embedding(doc["content"])
    doc["filename"] = doc['url'].split('/')[-1]

In [20]:
for doc in documents:
    print(doc["title"])

Medical Insurance - Hospitalization & Surgical Claim Form
Voluntary Health Insurance Scheme (VHIS) Claimable Amount Estimate / Preliminary Assessment
OGD/Colonoscopy Day Surgery or Hospital Admission Consultation Form
Group Medical Claim Form
Change of Contact Details Form
Change of Personal Particulars Form
Joint Owner Authorization Form
Customer Consent to Use / Provision of Personal Data in Direct Marketing
Change Premium Payment Mode/ Method
Beneficiary Designation Form
Change of Policyowner Personal Details Form
Risk Profile Questionnaire
Manulife Investment Plus 2 (Hong Kong) Investment Choice Table
Manulife Investment Plus 2 (Hong Kong) Application For Payout Distribution Investment Choice
Manulife Investment Plus 2 (Hong Kong) Investment Choice Services Form
Manulife Investment Plus 2 (Hong Kong) Policy Change Form


In [20]:
#upload documents to index
search_client = SearchClient(endpoint=search_endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(search_api_key))
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)


    index_client = SearchIndexClient(
    endpoint=search_endpoint, credential=AzureKeyCredential(search_api_key))

Upload of new document succeeded: True


In [21]:
documents[0].keys()

dict_keys(['@search.action', 'id', 'url', 'title', 'content', 'category', 'vector'])